In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **이미지분류경진대회**

In [ ]:
from PIL import Image
from matplotlib import pyplot as plt
import tensorflow as tf

In [ ]:
trainp="/kaggle/input/imge-classification-animal/animal-classification/train/"
testp="/kaggle/input/imge-classification-animal/animal-classification/test/"

In [ ]:
sub=pd.read_csv('/kaggle/input/imge-classification-animal/Sample_submission.csv')

# **(1) 데이터 불러오기/전처리: ImageDataGenerator (train, test)**

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
igen=ImageDataGenerator(rescale=1/255, validation_split=0.2,
                       # width_shift_range=0.1, height_shift_range=0.1, 
                        #zoom_range=0.1, horizontal_flip=True, vertical_flip=True,
                        #shear_range=0.1,brightness_range=(0.8,1.2),rotation_range=10
                       )
                       
tgen=ImageDataGenerator(rescale=1/255)

In [ ]:
for i in range (1,11):
    a=Image.open(trainp+'deer/deer_'+str(i)+".jpg")
    plt.imshow(a)
    print(a.size)
    plt.show()

데이터 변경을 위한 변수 설정

In [ ]:
bs=64
target_size1=256
isize=(target_size1,target_size1)
ishape=(target_size1,target_size1,3)

In [ ]:
traind=igen.flow_from_directory(trainp, target_size=isize, 
                                batch_size=bs, shuffle=True, class_mode ="sparse", subset="training")

In [ ]:
valid=igen.flow_from_directory(trainp, target_size=isize, 
                              batch_size=bs ,class_mode ="sparse", subset="validation", 
                              shuffle=False)

flow_from_directory(test_data)

In [ ]:
sub = pd.read_csv("/kaggle/input/imge-classification-animal/Sample_submission.csv")
test_load=sub.copy()
test_load['ID']=test_load['ID'].astype('str')
test_load['file']=testp+'test/'+test_load['ID']+'.jpg'

testd=tgen.flow_from_dataframe(test_load, x_col='file',batch_size=bs,  y_col=None, target_size=isize, class_mode=None, shuffle=False)

#testd=tgen.flow_from_directory(testp, target_size =isize, 
#                              batch_size=bs, class_mode = None, shuffle=False)

# **(3) CNN모델 라이브러리 불러오기**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# **(4) 사전에 훈련된 전이학습 Xception model 불러오기**

In [ ]:
base_model = tf.keras.applications.Xception(
    weights='imagenet',  #  ImageNet에 대해 사전 훈련된 가중치를 적용합니다. (Default)
    input_shape=ishape, 
    include_top=False) 
base_model.summary()

# **(5) 전이학습 모델+ CNN모델 설정: Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization**

In [ ]:
from tensorflow.keras.optimizers import RMSprop

base_model.trainable = False     #Freeze all layers 

inputs=tf.keras.Input(shape=ishape)

#scale_layer = tf.keras.layers.Rescaling(scale=1 /(0.5*target_size), offset=-1)
#x = scale_layer(inputs)


x = base_model(inputs, training=False)
x = tf.keras.layers.BatchNormalization()(x)
'''
x = tf.keras.layers.Conv2D(32, 3, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Conv2D(32, 3, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(32, 3, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
'''

x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)  # Regularize with dropout

x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.3)(x)

x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.3)(x)

x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.3)(x)

x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.3)(x)

x = tf.keras.layers.Dense(16, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)


outputs = tf.keras.layers.Dense(4, activation= 'softmax')(x)

model = tf.keras.Model(inputs, outputs)
model.summary()



# **(5) 모델 Compile**

In [ ]:


model.compile(optimizer=RMSprop(learning_rate=0.001),  #lr:learning_rate 보폭
            loss='sparse_categorical_crossentropy',
            metrics='acc')


# **(6) 모델 Fit**

In [ ]:
hist = model.fit(traind, epochs=30, validation_data=valid)

# **(7) loss와 val_loss 그래프 그리기**

In [ ]:
loss=hist.history['loss']
val_loss=hist.history['val_loss']
acc=hist.history['acc']
val_acc=hist.history['val_acc']
ep=np.arange(1,len(loss)+1)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8))
plt.plot(ep, loss, 'b', label='loss')
plt.plot(ep, val_loss, 'g', label="val_loss")
plt.title('loss')
plt.legend()

# **(8) acc와 val_acc 그래프 그리기**

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(ep, acc, 'b', label='acc')
plt.plot(ep, val_acc, 'g', label="val_acc")
plt.title('Accuracy')
plt.legend()

# **(9) 내보내기**

In [ ]:
result=model.predict(testd)
sub['Label']=result.argmax(1)
sub.to_csv('test_data_result.csv', index=False)

In [ ]:
#model.evaluate(testd)